In [1]:
import pandas as pd
# !pip install wrds
import wrds
import numpy as np
from stock_ret import stock_ret

In [2]:
data = pd.read_excel(r"G:\Resaerch_Data\erwin_collaboration\data_for_leo.xlsx")

In [3]:
data.head()

,obsnum,ibes_ticker,ibes_cname,SCORE,gvkey,permno,ticker,company_name,dummy_2000,dummy_2007,CUSIP_orig,CUSIP_new,permno_erwin,UPDATED_TICKER_new
0,1,HSM,HUSSMANN INTL INC,0.0,66293.0,85767.0,999,HUSSMANN INTERNATIONAL INC,1,0,44811010,NaN,85767.0,NaN
1,2,AAON,AAON INC,0.0,21542.0,76868.0,AAON,AAON INC,1,0,00036020,NaN,76868.0,NaN
2,3,AAIT,ADVANCED ANALOGIC TECHNOLOGIES I,0.0,163679.0,90854.0,AATI,ADVANCED ANALOGIC TECH,0,1,NaN,00752J108,NaN,AATI
3,4,ACAT,ARCTIC CAT INC,0.0,22461.0,76207.0,ACAT,ARCTIC CAT INC,0,1,NaN,39670104,NaN,ACAT
4,5,ACLR,ACCENT COLOR SCIENCES INC,0.0,64171.0,84288.0,ACLR,ACCENT COLOR SCIENCES,1,0,00430510,NaN,84288.0,NaN


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# check the tickers in the sample
tickers = data[['ibes_ticker','ticker','permno','gvkey']].sort_values(['ticker','permno'])

In [5]:
tickers = tickers.drop_duplicates('permno').dropna(subset=['permno'])
tickers.head()

,ibes_ticker,ticker,permno,gvkey
0,HSM,999,85767.0,66293.0
1,AAON,AAON,76868.0,21542.0
2,AAIT,AATI,90854.0,163679.0
3,ACAT,ACAT,76207.0,22461.0
4,ACLR,ACLR,84288.0,64171.0


In [6]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [7]:
tickers_l = tickers.permno.tolist()
tickers_l = ','.join((str(n) for n in tickers_l))

In [8]:
# read daily stock price from crsp between 2000-01-01 and 2016-12-21
query = "select * from crsp.dsf where date>= '2000-01-01' and date <= '2016-12-31' and permno IN (%s)" %tickers_l
stock_price = db.raw_sql(query)


In [9]:
stock_price.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx
0,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2000-01-03,20.500,21.875,20.7500,18002.0,0.012195,20.500,20.7500,9017.0,2.0,2.0,20.6875,24.0,0.012195
1,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2000-01-04,19.000,20.750,19.0000,4515.0,-0.084337,19.000,19.2500,9017.0,2.0,2.0,20.7500,18.0,-0.084337
2,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2000-01-05,19.000,20.125,20.1250,16535.0,0.059211,19.875,20.0625,9017.0,2.0,2.0,19.0000,36.0,0.059211
3,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2000-01-06,19.500,20.250,20.1875,25982.0,0.003106,20.125,20.2500,9017.0,2.0,2.0,19.7500,34.0,0.003106
4,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2000-01-07,19.875,20.250,20.2500,12640.0,0.003096,19.875,20.2500,9017.0,2.0,2.0,19.8750,16.0,0.003096


In [10]:
stock_price.shape
len(stock_price.permno.unique())

280

Query data from Compustats

In [11]:
company = db.get_table(library='comp', table='company')

In [12]:
company_df = pd.DataFrame(company)
company_df.shape

(52250, 39)

In [13]:
company.head()

,conm,gvkey,add1,add2,add3,add4,addzip,busdesc,cik,city,conml,costat,county,dlrsn,ein,fax,fic,fyrc,ggroup,gind,gsector,gsubind,idbflag,incorp,loc,naics,phone,prican,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,A & E PLASTIK PAK INC,001000,None,None,None,None,None,None,None,None,A & E Plastik Pak Inc,I,None,09,None,None,USA,12.0,None,None,None,None,D,None,USA,None,None,None,None,01,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
1,A & M FOOD SERVICES INC,001001,1924 South Utica Avenue,None,None,None,94104,None,0000723576,Tulsa,A & M Food Services Inc,I,None,01,73-0934497,None,USA,12.0,2530,253010,25,25301040,D,None,USA,722,None,None,None,01,5812,420.0,978.0,None,OK,0.0,None,1986-07-31,None
2,AAI CORP,001002,124 Industry Lane,None,None,None,21030-0126,"Textron Systems Corporation designs, develops,...",0001306124,Hunt Valley,AAI Corp,I,None,01,None,410-628-3215,USA,12.0,None,None,None,None,D,None,USA,None,410-666-1400,None,None,01,3825,230.0,940.0,None,MD,0.0,www.textronsystems.com,1977-03-31,None
3,A.A. IMPORTING CO INC,001003,7700 Hall Street,None,None,None,63125,"A.A. Importing Company, Inc. designs, manufact...",0000730052,St. Louis,A.A. Importing Co Inc,I,None,07,43-1232041,314-383-2608,USA,1.0,2550,255040,25,25504040,D,DE,USA,442110,314-383-8800,None,None,01,5712,449.0,976.0,None,MO,3.0,www.aaimporting.com,1992-04-30,None
4,AAR CORP,001004,"One AAR Place, 1100 North Wood Dale Road",None,None,None,60191,AAR Corp. provides products and services to co...,0000001750,Wood Dale,AAR Corp,A,None,None,36-2334820,630 227 2039,USA,5.0,2010,201010,20,20101010,D,DE,USA,423860,630 227 2000,None,None,01,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,None,1988-01-01


In [14]:
# gvkey_list
gvk_l = tickers[tickers['gvkey'].notna().tolist()].gvkey.tolist()

In [15]:
tickers[tickers['gvkey'].notna().tolist()].head()

,ibes_ticker,ticker,permno,gvkey
0,HSM,999,85767.0,66293.0
1,AAON,AAON,76868.0,21542.0
2,AAIT,AATI,90854.0,163679.0
3,ACAT,ACAT,76207.0,22461.0
4,ACLR,ACLR,84288.0,64171.0


In [16]:
gvk_l = [str(int(x)).zfill(6) for x in gvk_l]

In [17]:
# subset the company for the sample ticker list
company[company.gvkey.isin(gvk_l)].head()

,conm,gvkey,add1,add2,add3,add4,addzip,busdesc,cik,city,conml,costat,county,dlrsn,ein,fax,fic,fyrc,ggroup,gind,gsector,gsubind,idbflag,incorp,loc,naics,phone,prican,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
73,AVX CORP,001072,One AVX Boulevard,None,None,None,29644,"KYOCERA AVX Components Corporation, together w...",0000859163,Fountain Inn,AVX Corp.,I,None,01,33-0379007,None,USA,3.0,4520,452030,45,45203015,D,DE,USA,334416,864 967 2150,None,None,01,3670,220.0,925.0,B-,SC,0.0,www.kyocera-avx.com,2020-03-31,1973-03-01
106,ACME UNITED CORP,001104,"1 Waterview Drive, Suite 200",None,None,None,06484,Acme United Corporation supplies first aid and...,0000002098,Shelton,Acme United Corp,A,None,None,06-0236700,None,USA,12.0,2020,202010,20,20201060,D,CT,USA,332215,203 254 6060,None,None,01,3420,270.0,976.0,A-,CT,0.0,www.acmeunited.com,None,None
214,AIR PRODUCTS & CHEMICALS INC,001209,1940 Air Products Boulevard,None,None,None,18106-5500,"Air Products and Chemicals, Inc. provides atmo...",0000002969,Allentown,Air Products and Chemicals Inc.,A,None,None,23-1274455,None,USA,9.0,1510,151010,15,15101040,D,DE,USA,325120,610 481 4911,None,None,01,2810,160.0,970.0,A,PA,0.0,www.airproducts.com,None,1987-01-01
549,AMERICAN SCIENCE ENGINEERING,001554,829 Middlesex Turnpike,None,None,None,01821,"American Science and Engineering, Inc., togeth...",0000005768,Billerica,American Science and Engineering Inc.,I,None,01,04-2240991,978 262 0533,USA,3.0,2010,201010,20,20101010,D,DE,USA,334517,978 262 8700,None,None,01,3844,300.0,905.0,B-,MA,0.0,rapiscan-ase.com,2016-09-13,None
580,AMERICAN VANGUARD CORP,001585,4695 MacArthur Court,None,None,None,92660,"American Vanguard Corporation, through its sub...",0000005981,Newport Beach,American Vanguard Corp,A,None,None,95-2588080,949 260 1214,USA,12.0,1510,151010,15,15101030,D,DE,USA,325320,949 260 1200,None,None,01,2870,167.0,970.0,B,CA,0.0,www.american-vanguard.com,None,None


There are 17 companies missing gvkeys.

In [18]:
company.loc[0,'gvkey']
# since the data type of gvkey in company is string


'001000'

In [19]:
# convert it to int
# company['gvkey'] = company.gvkey.astype(int)

In [20]:
# company[company['gvkey'].isin(gvk_l)]

In [21]:
print(company.columns)

Index(['conm', 'gvkey', 'add1', 'add2', 'add3', 'add4', 'addzip', 'busdesc',
       'cik', 'city', 'conml', 'costat', 'county', 'dlrsn', 'ein', 'fax',
       'fic', 'fyrc', 'ggroup', 'gind', 'gsector', 'gsubind', 'idbflag',
       'incorp', 'loc', 'naics', 'phone', 'prican', 'prirow', 'priusa', 'sic',
       'spcindcd', 'spcseccd', 'spcsrc', 'state', 'stko', 'weburl', 'dldte',
       'ipodate'],
      dtype='object')


## Automating Data Query in Compustats from WRDS
https://davidkreitmeir.netlify.app/post/sql-wrds-query/

In [22]:
def build_query(dataset, **kwargs):
        """Build Query for WRDS database.
        Args:
            dataset (str): dataset
            start (str): Start of time period with format YYYY-MM-DD, default to None
            end (str): End of time period with format YYYY-MM-DD, default to None
            columns (str or list): Filter of one or a list of columns, default to None
            sic (str or list): Filter of one or a list of SIC, default to None
            sich (str or list): Filter of one or a list of historical SIC, default to None
            gvkey (str or list):  Filter of one or a list of Global Company Key, default to None
            tic (str or list):  Filter of one or a list of ticker symbols, default to None
            cusip (str or list):  Filter of one or a list of CUSIP symbols, default to None
            limit (int): Return only a number of return records, defualt to None
        Returns:
            (str) with query
        """
        columns = kwargs.get("columns", None)
        sic = kwargs.get("sic", None)
        sich = kwargs.get("sich", None)
        start = kwargs.get("start", None)
        end = kwargs.get("end", None)
        gvkey = kwargs.get("gvkey", None)
        tic = kwargs.get("tic", None)
        cusip = kwargs.get("cusip", None)
        limit = kwargs.get("limit", None)

        if columns is not None:
            columns_filter = ",".join([x for x in columns])
        else:
            columns_filter = "*"

        date_filter = ""
        if start and end is not None:
            date_filter = "datadate between {} and {}".format("'" + start + "'" , "'" + end + "'")

        sic_filter = ""
        if sic is not None:
            sic_filter = "sic in ({}) ".format(
                ",".join(["'" + x + "'" for x in sic])
            )

        sich_filter = ""
        if sich is not None:
            sich_filter = "sich in ({}) ".format(
                ",".join(["'" + x + "'" for x in sich])
            )

        gvkey_filter = ""
        if gvkey is not None:
            gvkey_filter = "gvkey in ({}) ".format(
                ",".join(["'" + x + "'" for x in gvkey])
            )

        tic_filter = ""
        if tic is not None:
            tic_filter = "tic in ({}) ".format(
                ",".join(["'" + x + "'" for x in tic])
            )

        cusip_filter = ""
        if cusip is not None:
            cusip_filter = "cusip in ({}) ".format(
                ",".join(["'" + x + "'" for x in cusip])
            )

        # FIRST INSTANCE WITHOUT AND THEN SEQUENTIALLY ADD FILTERS (if non empty)
        filters = [date_filter, cusip_filter, tic_filter, gvkey_filter, sic_filter]
        filters_list = ""
        if any(filters):
            filters_list = " and ".join([x for x in filters if x != ""])
            filters_list = "where " + filters_list

        limit_number = ""
        if limit:
            limit_number += "LIMIT {} ".format(limit)

        cmd = (
            "select "
            + columns_filter
            + " from {} ".format(dataset)
            + filters_list
            + limit_number
            )

        print(cmd)
        return(cmd)

In [23]:
# query data from compustat - comp.funda
columns = ["gvkey","cik","fyearq", "datadate", "indfmt", "consol", "popsrc", "datafmt", "tic", "cusip", "conm",
           "fyr", "datacqtr", "datafqtr", "epsfxq", "uniamiq" ,"atq", "mkvaltq", "prccq", "cshoq", "cshprq", "seqq"]
columns_a = ["gvkey","cik","fyear", "indfmt", "consol", "popsrc", "datafmt", "tic","cusip", "conm",
           "fyr", "datacqtr", "datafqtr", "epsfx", "uniami", "at", "mkvalt", "prcc", "cshi", "cshpr"]
query_compq = build_query(dataset = 'comp.fundq', 
                          columns = columns,
                          gvkey = gvk_l,
                          start = "2000-01-01",
                          end = "2016-12-31")
query_compa = build_query(dataset = 'comp.funda',
                          columns = columns_a,
                          gvkey = gvk_l,
                          start = "2000-01-01",
                          end = "2016-12-31")

select gvkey,cik,fyearq,datadate,indfmt,consol,popsrc,datafmt,tic,cusip,conm,fyr,datacqtr,datafqtr,epsfxq,uniamiq,atq,mkvaltq,prccq,cshoq,cshprq,seqq from comp.fundq where datadate between '2000-01-01' and '2016-12-31' and gvkey in ('066293','021542','163679','022461','064171','137602','001104','061427','001632','031455','064638','062990','025180','066691','029968','027960','024302','001820','014324','001704','110039','138741','012379','001651','264387','001209','014282','027997','025355','028853','061996','163628','064840','001554','030584','029274','003333','062884','139662','001585','027925','025641','001072','065196','002058','002044','029284','002124','001981','002393','064680','061596','011914','001968','005133','002189','024363','013119','060898','002802','127455','011805','113361','002537','003619','133726','062516','160203','063617','012390','062221','112092','003358','013623','105936','136804','003622','065850','002577','064557','029705','016710','062913','121153','003971','0

In [24]:
comp_fundq_s = db.raw_sql(query_compq)
comp_fundq_s.sort_values(by = ['gvkey', 'datadate'], inplace = True)

In [25]:
comp_fundq_s.head()

,gvkey,cik,fyearq,datadate,indfmt,consol,popsrc,datafmt,tic,cusip,conm,fyr,datacqtr,datafqtr,epsfxq,uniamiq,atq,mkvaltq,prccq,cshoq,cshprq,seqq
28,001072,0000859163,1999.0,2000-03-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q1,1999Q4,0.80,None,1308.331,NaN,75.8125,87.246,87.185,982.021
295,001072,0000859163,2000.0,2000-06-30,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q2,2000Q1,0.68,None,1469.237,NaN,22.9375,175.080,174.914,1102.252
569,001072,0000859163,2000.0,2000-09-30,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q3,2000Q2,0.89,None,1601.888,NaN,26.0625,174.654,174.754,1228.972
871,001072,0000859163,2000.0,2000-12-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q4,2000Q3,0.94,None,1802.707,NaN,16.3750,174.657,174.656,1401.432
1079,001072,0000859163,2000.0,2001-03-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2001Q1,2000Q4,0.71,None,1885.098,NaN,17.2600,174.702,174.688,1505.034


In [26]:
len(comp_fundq_s['gvkey'].unique())

271

<ol>
    <p> Variables Names</p>
    <li>https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/</li></ol>

In [27]:
db.describe_table(library='comp', table='fundq').head()

Approximately 1935985 rows in comp.fundq.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),None
1,datadate,True,DATE,None
2,fyearq,True,DOUBLE_PRECISION,None
3,fqtr,True,DOUBLE_PRECISION,None
4,fyr,True,DOUBLE_PRECISION,None


In [28]:
# # comp_fundq = db.raw_sql("select * from comp.fundq")
# #query for the whole table - comp.fundq
# df = db.get_table(library = 'comp', table = 'fundq')
# df.shape
# # write df to pickle
# # df.to_pickle(r"G:\Resaerch_Data\compustat\comp_Q\comp_Q.pkl")
# import pickle

# with open(r"G:\Resaerch_Data\compustat\comp_Q\comp_Q.pkl","rb") as f:
#     comp_fundq = pickle.load(f)
# comp_fundq.head()
# comp_fundq.loc[0,'gvkey']
# comp_fundq.shape
# # check the number of unique gvkeys
# len(comp_fundq['gvkey'].unique())

In [29]:
columns_ = ["indfmt", "consol", "popsrc", "datafmt"]
for col in columns_:
    print(comp_fundq_s[col].unique())

['INDL']
['C' 'R']
['D']
['STD']


#### ROA - return on assets is defined as income before extraordinary items divided by lagged total assets.
$$ ROA_{i,t}= \frac{UNIAMIQ_{i,t}}{ATQ_{i,t−1}} $$
$$ UNIAMIQ_{i,t} = \sum_{t-3}^{t}{EPSFXQ_{i,t}} \times {CSHPRQ_{i,t}} $$
###### UNIAMIQ  : This item represents the income after all operating and nonoperating income, expense, subsidiary preferred dividends, and minority interest but before preferred dividends, preference dividends, and common dividends. It is also stated before all extraordinary items that are listed in the company's public records as being net of taxes. UNIAMI is annulized net income beore Extraordinary Items. 12-month moving average.

<ol>
    <li>Annual ROA: </li>
    <ul><li>return on assets is defined as income before extraordinary items divided by lagged total assets.</li></ul>
<!--     <li>Volatility: 
        <p>$$ ROAqVolatility_{i,y}=\sqrt{\frac{\sum_{q=1}^{q=4} (ROA_{i,y,q} - \frac{\sum_{q=1}^{q=4}}{4}{4}}$$ </p>
    </li> -->
    <li>Volatility - annual standard deviation quarterly ROA</li>
    <ul><li>Adams et al. 2005 (p1419) compute the ROA and Q for each firm using yearly(ROA and Q) values from 1992 to 1999!</li></ul>
<!--     <p><img src=".\formula\roa.png" alt="Annual ROA" title="ROA"></p> -->
</ol>


#### MTB : Market to Book Ratio (Calendar Year)

$$ MTB_{i,y}= \frac{\sum_{q=1}^{q=4}MKVALTQ_{i,y,q}}{\sum_{q=1}^{q=4}ATQ_{i,y,q}}$$
$$ MKVALTQ_{i,y,q} = PRCCQ_{i,y,q} \times CSHOQ_{i,y,q} $$


#### TobinQ : Tobin's Q is defined as the ratio between the market value of the firm (MV) over the replacement cost of its assets (RC). Tobin’s Q reflects how much the market values the company as a whole, including valuable unmeasured assets, relative to its existing exchangeable assets.

$$ Tobin's Q = \frac{ATQ_{i,t} - SEQQ_{i,t} + MKVALTQ{i,t}}{ATQ_{i,t}} $$


In [30]:
def fillna_col(row, col_na):
    """
    The input1 row is the row of the dataframe
    The input2 should be column names which are missing vlaues, like 'UNAMIIQ' or 'MKVALTQ'
    """
    if pd.isna(row[col_na]) and col_na =='uniamiq':
        return row['epsfxq']*row['cshprq']
    elif pd.isna(row[col_na]) and col_na =='mkvaltq':
        return row['prccq']*row['cshoq']
    else:
        return np.nan
        
def fillna_unamiiq(df):
    """The input1 should be the dataframe for a certain gvkey"""
    df['temp'] = df.apply(lambda row: fillna_col(row, 'uniamiq'), axis = 1)
    # rolling window sum the temp column
    df.sort_values(by = 'datadate', inplace = True)
    df['temp_sum'] = df['temp'].rolling(4).sum()
    df['uniamiq'].fillna(df['temp_sum'], inplace = True)
    df.drop(columns = ['temp','temp_sum'], inplace = True)
    # print(df)
    return df

def fillna_mkvaltq(df):
    """The input1 should be the dataframe for a certain gvkey"""
    df['temp'] = df.apply(lambda row: fillna_col(row, 'mkvaltq'), axis = 1)
    df.sort_values(by = 'datadate', inplace = True)
    df['mkvaltq'].fillna(df['temp'], inplace = True)
    df.drop(columns = 'temp', inplace = True)
    return df

def roa(row, ni, atq_lag_1):
    '''The 1st input should be the row of the dataframe,
    The 2nd input should be the column name of the net income, like 'ni'-net income before extraordinary items
    The 3rd input should be the column name of the lag_1 of the total assets, like 'atq_lag_1
    The output is the return on assets
    '''
    if pd.isna(row[atq_lag_1]):
        return np.nan
    else:
        return row[ni]/row[atq_lag_1]

def lag_1(df_col):
    """The input should be data series, like df['atq']"""
    return df_col.shift(1)

def mtb(row, mkvaltq, atq):
    """The 1st input should be the row of the dataframe,
    The 2nd input should be the column name of the market value of equity, like 'mkvaltq'
    The 3rd input should be the column name of the total assets, like 'atq'"""
    if pd.isna(row[atq]):
        return np.nan
    elif row[mkvaltq] ==0 or pd.isna(row[mkvaltq]):
        row[mkvaltq] = row['prccq']*row['cshoq']
        return row[mkvaltq]/row[atq]
    else:
        return row[mkvaltq]/row[atq]

def tobinQ(row, atq, seqq, mkvaltq):
    """The 1st input should be the row of the dataframe,
    The 2nd input should be the column name of the total assets, like 'atq'
    The 3rd input should be the column name of the total equity, like 'seqq'"""
    if pd.isna(row[atq]) or row[atq]==0:
        return np.nan
    else:
        return (row[atq] - row[seqq] + row[mkvaltq])/row[atq]

def rolling_mean(df_col, window):
    """The input should be data series, like df['mtb']"""
    return df_col.rolling(window).mean()

def annulized_sum(df, key_col, year_col):
    """The input should be time series data, like df['mkvaltq'], quarterly"""
    key_col_ann_sum = key_col + '_ann_sum'
    temp = df.groupby(year_col)[key_col].sum().reset_index().rename(columns={key_col:key_col_ann_sum})
    df = df.merge(temp, on=key_col, how='left')
    return df

def rolling_std(df_col, window):
    """The input should be data series, like df['atq']"""
    return df_col.rolling(window).std()

def annulized_mean(df_panel, key_col, year_col):
    """The input should be data series, like df['mtb']"""
    key_col_ann_mean = key_col + '_ann_mean'
    df_panel[key_col_ann_mean] = df_panel.groupby(year_col)[key_col].sum().reset_index().rename(columns={key_col:key_col_ann_mean})
    return df_panel

def annulized_vol(df, key, window, min_periods=2, year_col='fyearq'):
    """The input should be data series, like df['mtb']
    the cols is the list of the columns to be calculated,
    input1: df_panel, the dataframe of the panel data
    input2: key, the column name of the key inputs, like roa, mtb, etc.
    input3: window, the window of the rolling
    input4: min_periods, the min_periods of the rolling
    input5: year_col, in compustats, it usually called fyearq, which has year, like 2016, 2017, etc."""
    key_std = key + '_ann_std'
    key_mean = key + '_ann_mean'
    temp = df.groupby(year_col)[key].mean().reset_index().rename(columns={key:key_mean})
    temp[key_std] = temp[key_mean].rolling(window, min_periods).std()
    df = pd.merge(df, temp, on = 'fyearq', how = 'left')
    return df

def panel_analysis(df_input,key_col):
    ''' the function is to analysis the panel data,
    the input is the dataframe of the panel data
    the output is the dataframe of the analysis result
    the key_col is the column name of the key, like gvkey, cik, etc.
    '''
    # process bar
    from tqdm import tqdm
    # for each key, run the algorithm
    loop_key = df_input[key_col].unique().tolist()
    df_output = pd.DataFrame()
    for key in tqdm(loop_key):
        # get the data of the key
        df_key = df_input[df_input[key_col] == key]
        # sort the data by the date
        df_key = df_key.sort_values(by = 'datadate').reset_index(drop = True)
        # get the lag 1 of the atq
        df_key['atq_lag_1'] = lag_1(df_key['atq'])
        # fillna if uniamiq is nan
        df_key = fillna_unamiiq(df_key)
        # fillna if mkvaltq is nan
        df_key = fillna_mkvaltq(df_key)
        # get the roa
        df_key['roa'] = df_key.apply(lambda x: roa(x, 'uniamiq', 'atq_lag_1'), axis=1)
        # get the mtb
        df_key['mtb'] = df_key.apply(lambda x: mtb(x, 'mkvaltq', 'atq'), axis=1)
        # get the tobinQ
        df_key['tobinQ'] = df_key.apply(lambda x: tobinQ(x, 'atq', 'seqq', 'mkvaltq'), axis=1)
        # get the annualized standard deviation of roa, quarterly
        df_key = annulized_vol(df_key, 'roa', 5, 2, 'fyearq')
        # append the df_key to the df_output
        df_output = pd.concat([df_output,df_key], axis=0)
    return df_output.reset_index(drop = True)

In [31]:
comp_fundq_s = comp_fundq_s[(comp_fundq_s['indfmt']=='INDL')&(comp_fundq_s['consol']=='C')]

In [32]:
print(comp_fundq_s.shape)
print(pd.isna(comp_fundq_s['atq']).sum())

(12829, 22)
154


In [33]:
# copy the comp_fundq_s to a new dataframe as input for compute_dv
df = comp_fundq_s.copy()

def compute_dv(df):
    '''Intput is comp_fundq_s'''
    # drop colums that are not needed
    df = df.drop(columns=['indfmt', 'consol', 'popsrc', 'datafmt'])
    # group by gvkey and compute the dv
    gvks = df['gvkey'].unique().tolist()
    # run compute_dv for each gvkey
    df_output = panel_analysis(df,'gvkey')
    return df_output

In [34]:
comp_fundq_s = comp_fundq_s.sort_values(['gvkey','datadate']).reset_index(drop=True) # sort by gvkey and datadate
comp_fundq_s.head()

,gvkey,cik,fyearq,datadate,indfmt,consol,popsrc,datafmt,tic,cusip,conm,fyr,datacqtr,datafqtr,epsfxq,uniamiq,atq,mkvaltq,prccq,cshoq,cshprq,seqq
0,001072,0000859163,1999.0,2000-03-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q1,1999Q4,0.80,None,1308.331,NaN,75.8125,87.246,87.185,982.021
1,001072,0000859163,2000.0,2000-06-30,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q2,2000Q1,0.68,None,1469.237,NaN,22.9375,175.080,174.914,1102.252
2,001072,0000859163,2000.0,2000-09-30,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q3,2000Q2,0.89,None,1601.888,NaN,26.0625,174.654,174.754,1228.972
3,001072,0000859163,2000.0,2000-12-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2000Q4,2000Q3,0.94,None,1802.707,NaN,16.3750,174.657,174.656,1401.432
4,001072,0000859163,2000.0,2001-03-31,INDL,C,D,STD,AVX,002444107,AVX CORP,3.0,2001Q1,2000Q4,0.71,None,1885.098,NaN,17.2600,174.702,174.688,1505.034


In [35]:
df_output = compute_dv(df)

100%|██████████| 270/270 [00:03<00:00, 76.50it/s]


In [36]:
df_output.shape

(12829, 24)

In [37]:
df_output.head(20)

,gvkey,cik,fyearq,datadate,tic,cusip,conm,fyr,datacqtr,datafqtr,epsfxq,uniamiq,atq,mkvaltq,prccq,cshoq,cshprq,seqq,atq_lag_1,roa,mtb,tobinQ,roa_ann_mean,roa_ann_std
0,001072,0000859163,1999.0,2000-03-31,AVX,002444107,AVX CORP,3.0,2000Q1,1999Q4,0.80,NaN,1308.331,6614.337375,75.8125,87.246,87.185,982.021,NaN,NaN,5.055554,5.304963,NaN,NaN
1,001072,0000859163,2000.0,2000-06-30,AVX,002444107,AVX CORP,3.0,2000Q2,2000Q1,0.68,NaN,1469.237,4015.897500,22.9375,175.080,174.914,1102.252,1308.331,NaN,2.733322,2.983101,0.314752,NaN
2,001072,0000859163,2000.0,2000-09-30,AVX,002444107,AVX CORP,3.0,2000Q3,2000Q2,0.89,NaN,1601.888,4551.919875,26.0625,174.654,174.754,1228.972,1469.237,NaN,2.841597,3.074395,0.314752,NaN
3,001072,0000859163,2000.0,2000-12-31,AVX,002444107,AVX CORP,3.0,2000Q4,2000Q3,0.94,508.39722,1802.707,2860.008375,16.3750,174.657,174.656,1401.432,1601.888,0.317374,1.586508,1.809103,0.314752,NaN
4,001072,0000859163,2000.0,2001-03-31,AVX,002444107,AVX CORP,3.0,2001Q1,2000Q4,0.71,562.67770,1885.098,3015.356520,17.2600,174.702,174.688,1505.034,1802.707,0.312129,1.599575,1.801190,0.314752,NaN
5,001072,0000859163,2001.0,2001-06-30,AVX,002444107,AVX CORP,3.0,2001Q2,2001Q1,0.17,473.45235,1810.762,3672.816000,21.0000,174.896,174.801,1526.858,1885.098,0.251155,2.028326,2.185113,0.118581,0.138714
6,001072,0000859163,2001.0,2001-09-30,AVX,002444107,AVX CORP,3.0,2001Q3,2001Q2,-0.16,289.93505,1769.424,2840.579300,16.2700,174.590,174.914,1502.547,1810.762,0.160118,1.605369,1.756197,0.118581,0.138714
7,001072,0000859163,2001.0,2001-12-31,AVX,002444107,AVX CORP,3.0,2001Q4,2001Q3,-0.04,118.77961,1714.589,4116.289870,23.5900,174.493,174.470,1485.862,1769.424,0.067129,2.400744,2.534145,0.118581,0.138714
8,001072,0000859163,2001.0,2002-03-31,AVX,002444107,AVX CORP,3.0,2002Q1,2001Q4,-0.01,-6.99436,1691.599,3657.778260,20.9400,174.679,174.549,1476.035,1714.589,-0.004079,2.162320,2.289752,0.118581,0.138714
9,001072,0000859163,2002.0,2002-06-30,AVX,002444107,AVX CORP,3.0,2002Q2,2002Q1,0.01,-34.96313,1721.751,2854.124740,16.3300,174.778,174.740,1514.480,1691.599,-0.020669,1.657687,1.778071,-0.007200,0.162253


In [38]:
pd.isnull(df_output).sum()

gvkey              0
cik              131
fyearq             0
datadate           0
tic                0
cusip              0
conm               0
fyr                0
datacqtr          10
datafqtr           9
epsfxq           151
uniamiq          977
atq              154
mkvaltq          334
prccq            293
cshoq            214
cshprq           119
seqq             160
atq_lag_1        423
roa             1046
mtb              338
tobinQ           344
roa_ann_mean     465
roa_ann_std     1409
dtype: int64

$$ Ret_{t} = \frac{Price_{t_i} - Price_{t_{i-1}} + div_{t}}{Price_{t_{i-1}}} $$

$$ Ret_{t} = \frac{P_{t_i} - P_{t_0} + Div_{t}}{P_{t_0}} $$

$$ Daily\_log\_Ret_{t} = ln(1 + \frac{Price_{t_i} - Price_{t_{i-1}}}{Price_{t_{i-1}}}) $$

In [39]:
# tic = 'AAPL'
# investinfo = {'name': tic, }
# obj = stock_ret(tic, df_output)
# obj.comp_daily_returns()

## 3. Automating Data Query from CRSP

In [40]:
# query data from crsp - 
columns_crsp = ["cusip", "permno", "permco",  "shrcd", "issuno", "exchcd", "hexcd", "siccd", "hsiccd", "hsicmg", "nameendt",
                "tsymbol", "naics", "primexch", "trdstat", "secstat", "prc" ,"vol", "openprc", "askhi", "bidlo", "bid", "ask", "numtrd", "ret",
                "retx", "shrout", "shrflg", "shrenddt", "dlstcd", "nwperm", "nextdt", "dlamt", "dlprc", "dlpdt", "dlretx", "distcd", "divamt",
                "facpr", "facshr", "dclrdt", "rcrddt", "paydt", "acperm", "accomp", "trtscd", "nmsind", "mmcnt", "nsdinx", "vwretd", "vwretx",
                "ewretd", "ewretx", "sprtrn"]
""" 
    primexch: primary exchange code
    datafqtr: data frequency
    shrcd: share code, for example, 10 is common stock 
    epsfxq: earnings per share, basic
    trdstt: trading status
    secstat: security status
    dlamt: amount after delisting
    dlpdt: date of delisting
    dlretx: delisting return excluding dividends
"""
query_crspa = build_query(dataset = 'crsp.dsf', 
                          columns = columns_crsp,
                          gvkey = gvk_l,
                          start = "2000-01-01",
                          end = "2016-12-31")

select cusip,permno,permco,shrcd,issuno,exchcd,hexcd,siccd,hsiccd,hsicmg,nameendt,tsymbol,naics,primexch,trdstat,secstat,prc,vol,openprc,askhi,bidlo,bid,ask,numtrd,ret,retx,shrout,shrflg,shrenddt,dlstcd,nwperm,nextdt,dlamt,dlprc,dlpdt,dlretx,distcd,divamt,facpr,facshr,dclrdt,rcrddt,paydt,acperm,accomp,trtscd,nmsind,mmcnt,nsdinx,vwretd,vwretx,ewretd,ewretx,sprtrn from crsp.dsf where datadate between '2000-01-01' and '2016-12-31' and gvkey in ('066293','021542','163679','022461','064171','137602','001104','061427','001632','031455','064638','062990','025180','066691','029968','027960','024302','001820','014324','001704','110039','138741','012379','001651','264387','001209','014282','027997','025355','028853','061996','163628','064840','001554','030584','029274','003333','062884','139662','001585','027925','025641','001072','065196','002058','002044','029284','002124','001981','002393','064680','061596','011914','001968','005133','002189','024363','013119','060898','002802','127455','0118

#### Possible Columns in CRSP
["cusip","ncusip","comnam", "ticker", "permco", "shrcd", "issuno", "exchcd", "hexcd", "siccd", "hsiccd", "hsicmg", "nameendt",
                "tsymbol", "naics", "primexch", "trdstat", "secstat", "prc" ,"vol", "openprc", "askhi", "bidlo", "bid", "ask", "numtrd", "ret",
                "retx", "shrout", "shrflg", "shrenddt", "dlstcd", "nwperm", "nextdt", "dlamt", "dlprc", "dlpdt", "dlretx", "distcd", "divamt",
                "facpr", "facshr", "dclrdt", "rcrddt", "paydt", "acperm", "accomp", "trtscd", "nmsind", "mmcnt", "nsdinx", "vwretd", "vwretx",
                "ewretd", "ewretx", "sprtrn"]

In [41]:
# crsp_s = db.raw_sql(query_crspa)

In [42]:
db.get_table(library='crsp', table='dsf', obs=5)

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx
0,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-19,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
1,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-22,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
2,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-23,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
3,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-24,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
4,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-25,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0


#### List all tables in CRSP

```python code:
    db.list_tables(library='crsp')
```

In [43]:
db.get_table(library='crsp', table='dsf', obs=5)

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx
0,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-19,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
1,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-22,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
2,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-23,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
3,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-24,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0
4,93070510,12281.0,9974.0,13303.0,3.0,6020.0,1996-04-25,4.5,5.0,-4.75,0.0,0.0,4.5,5.0,4810.0,1.694879,1.694879,None,0.0,0.0


In [47]:
db.describe_table(library='crsp', table='wrds_dsfv2_query').to_csv(r"..\output\crsp_dsfv2_v_description.csv")

Approximately 101741688 rows in crsp.wrds_dsfv2_query.


In [ ]:
# db.get_table(library='crsp', table='riskfree', obs=5)